# Invert RedFlag Questions

This notebook helps you identify and invert YES/NO questions where "Yes" should be a bad answer.

**Current Logic:**
- "Yes" = 7 points (bad/toxic)
- "No" = 0 points (good/not toxic)

**Goal:**
- All questions should be phrased so that "Yes" = bad answer
- If a question currently has "Yes" = good, we need to invert the question text

**Example:**
- ❌ Bad: "Does he have proper grammar?" (Yes = good, but Yes = 7 points)
- ✅ Good: "Does he have poor grammar?" (Yes = bad, Yes = 7 points)


In [2]:
import pandas as pd
import os
import sys

# Add project root to Python path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src.utils.constants import CSV_SEPARATOR

# Configuration
USE_DYNAMODB = False  # Set to True to also update DynamoDB
CSV_FILE = os.path.join(project_root, "data", "RedFlagQuestions.csv")

# Load questions
df = pd.read_csv(CSV_FILE, sep=CSV_SEPARATOR)

# Filter YES/NO questions
yes_no_questions = df[df["Scoring"] == "YES/NO"].copy()

print(f"Total questions: {len(df)}")
print(f"YES/NO questions: {len(yes_no_questions)}")
print("\n" + "="*80)
print("YES/NO Questions to Review:")
print("="*80)

for idx, row in yes_no_questions.iterrows():
    print(f"\nID: {row['ID']} | RedFlag_ID: {row['RedFLag_ID']}")
    print(f"Category: {row['Category_Name']}")
    print(f"TR: {row['Question_TR']}")
    print(f"EN: {row['Question_EN']}")
    print("-" * 80)


Total questions: 75
YES/NO questions: 24

YES/NO Questions to Review:

ID: 2 | RedFlag_ID: 2
Category: Sosyal Kimlik
TR: Prestijli ve köklü bir lise de mi okudu? Örneğin: Galatasaray, Kabataş, Üsküdar Amerikan, Kadıköy Anadolu, İstanbul Erkek, Robert..
EN: Did he attend a prestigious and well-established high school? (e.g., Galatasaray, Kabataş, Üsküdar American, Kadıköy Anadolu, Istanbul Erkek, Robert, etc.)
--------------------------------------------------------------------------------

ID: 4 | RedFlag_ID: 4
Category: Sosyal Kimlik
TR: Mesleği şunlardan birisi mi?: Doktor, Avukat, Dişhekimi, PT(personal trainer), Akademisyen, Sanatçı
EN: Is his profession one of the following? Doctor, Lawyer, Dentist, Personal Trainer, Academic, Artist
--------------------------------------------------------------------------------

ID: 5 | RedFlag_ID: 5
Category: Sosyal Kimlik
TR: De/da'ları, gerektiğinde ki'yi ayrı yazabiliyor mu?
EN: Do they have the decency to type you messages with following pr

## Step 1: Identify Questions to Invert

Review the questions above and identify which ones need to be inverted.

**Questions that need inversion:**
- Questions where "Yes" currently means something GOOD
- Example: "Does he have proper grammar?" → Should be "Does he have poor grammar?"

**Questions that DON'T need inversion:**
- Questions where "Yes" already means something BAD
- Example: "Does he live with his family?" (Yes = bad, which is correct)


In [ ]:
# List of question IDs that need to be inverted
# Add the IDs of questions where "Yes" should be bad but currently "Yes" is good
QUESTIONS_TO_INVERT = [
    5,  # ID 5: Grammar question - "Does he have proper grammar?" → "Does he have poor grammar?"
    73,  # ID 73: Recent breakup - "Did his last relationship end less than six months ago?" → "Did his last relationship end within six months before you started dating?"
    # Add more question IDs here if needed
]

print(f"Questions to invert: {QUESTIONS_TO_INVERT}")

# Show the questions that will be inverted
if QUESTIONS_TO_INVERT:
    print("\n" + "="*80)
    print("Questions that will be inverted:")
    print("="*80)
    
    for q_id in QUESTIONS_TO_INVERT:
        row = df[df["ID"] == q_id].iloc[0]
        print(f"\nID: {q_id}")
        print(f"Current TR: {row['Question_TR']}")
        print(f"Current EN: {row['Question_EN']}")
        print("-" * 80)
else:
    print("\nNo questions selected for inversion.")
    print("Please add question IDs to QUESTIONS_TO_INVERT list above.")


Questions to invert: [5, 73]

Questions that will be inverted:

ID: 5
Current TR: De/da'ları, gerektiğinde ki'yi ayrı yazabiliyor mu?
Current EN: Do they have the decency to type you messages with following proper grammar rules ?
--------------------------------------------------------------------------------

ID: 73
Current TR: Son ilişkisi biteli en az bir 6 ay oldu mu?
Current EN: Did his last relationship end less than six months ago?
--------------------------------------------------------------------------------


## Step 2: Invert Question Text

This function helps invert question text. You can manually edit the questions or use helper functions.

**Common inversion patterns:**
- "Does he have X?" → "Does he lack X?" or "Does he not have X?"
- "Is he X?" → "Is he not X?" or "Is he lacking X?"
- "Can he X?" → "Can he not X?" or "Is he unable to X?"


In [4]:
def invert_question_text(text: str, language: str = "EN") -> str:
    """
    Helper function to invert question text.
    This is a basic helper - you may need to manually edit some questions.
    """
    # Common patterns for English
    if language == "EN":
        # Add negation
        if text.startswith("Does he have"):
            return text.replace("Does he have", "Does he lack")
        elif text.startswith("Does he"):
            return text.replace("Does he", "Does he not")
        elif text.startswith("Is he"):
            return text.replace("Is he", "Is he not")
        elif text.startswith("Can he"):
            return text.replace("Can he", "Is he unable to")
    
    # Common patterns for Turkish
    elif language == "TR":
        # Add negation
        if "yapabiliyor mu?" in text or "yapabilir mi?" in text:
            return text.replace("yapabiliyor mu?", "yapamıyor mu?").replace("yapabilir mi?", "yapamaz mı?")
        elif "var mı?" in text:
            return text.replace("var mı?", "yok mu?").replace("Var mı?", "Yok mu?")
    
    # If no pattern matches, return original (you'll need to edit manually)
    return text

# Apply inversion to selected questions
for q_id in QUESTIONS_TO_INVERT:
    row_idx = df[df["ID"] == q_id].index[0]
    
    # Invert English question
    current_en = df.loc[row_idx, "Question_EN"]
    inverted_en = invert_question_text(current_en, "EN")
    
    # Invert Turkish question
    current_tr = df.loc[row_idx, "Question_TR"]
    inverted_tr = invert_question_text(current_tr, "TR")
    
    # Update the dataframe
    df.loc[row_idx, "Question_EN"] = inverted_en
    df.loc[row_idx, "Question_TR"] = inverted_tr
    
    print(f"\nID {q_id} - Inverted:")
    print(f"  EN: {current_en} → {inverted_en}")
    print(f"  TR: {current_tr} → {inverted_tr}")

print("\n" + "="*80)
print("Review the inversions above. If they look incorrect, edit them manually.")
print("="*80)



ID 5 - Inverted:
  EN: Do they have the decency to type you messages with following proper grammar rules ? → Do they have the decency to type you messages with following proper grammar rules ?
  TR: De/da'ları, gerektiğinde ki'yi ayrı yazabiliyor mu? → De/da'ları, gerektiğinde ki'yi ayrı yazabiliyor mu?

ID 73 - Inverted:
  EN: Did his last relationship end less than six months ago? → Did his last relationship end less than six months ago?
  TR: Son ilişkisi biteli en az bir 6 ay oldu mu? → Son ilişkisi biteli en az bir 6 ay oldu mu?

Review the inversions above. If they look incorrect, edit them manually.


## Step 3: Manual Edits (if needed)

If the automatic inversion didn't work correctly, you can manually edit questions here.


In [5]:
# Manual edits - uncomment and modify as needed
# Example:
df.loc[df["ID"] == 5, "Question_EN"] = "Does he have poor grammar?"
df.loc[df["ID"] == 5, "Question_TR"] = "De/da'ları ve gerektiğinde ki'yi ayrı yazamaz?"
# Add your manual edits here:

df.loc[df["ID"] == 73, "Question_EN"] = "Did his last relationship end within six months before you started dating?"
df.loc[df["ID"] == 73, "Question_TR"] = "Son ilişkisi, sizin ilişkiniz başlamadan önceki 6 ay içinde mi bitti?"

## Step 4: Save to CSV


In [6]:
# Save updated questions to CSV
df.to_csv(CSV_FILE, sep=CSV_SEPARATOR, index=False)
print(f"✅ Updated questions saved to {CSV_FILE}")


✅ Updated questions saved to c:\Users\Pelin\Documents\Projects\runawayguys\data\RedFlagQuestions.csv


## Step 5: Update DynamoDB (if using DynamoDB)


In [ ]:
if USE_DYNAMODB:
    from src.adapters.database.database_handler import DatabaseHandler
    
    db_handler = DatabaseHandler(db_read_allowed=True, db_write_allowed=True)
    
    # Update each inverted question in DynamoDB
    for q_id in QUESTIONS_TO_INVERT:
        row = df[df["ID"] == q_id].iloc[0]
        
        # Prepare update dictionary
        update_dict = {
            "Question_TR": row["Question_TR"],
            "Question_EN": row["Question_EN"],
        }
        
        # Update in DynamoDB
        db_handler.update_record("RedFlagQuestions", {"ID": int(q_id)}, update_dict)
        print(f"✅ Updated question ID {q_id} in DynamoDB")
    
    db_handler.close()
    print("\n✅ All questions updated in DynamoDB")
else:
    print("⚠️  DynamoDB update skipped (USE_DYNAMODB = False)")


[INFO] Running locally, looking for credentials in file
[OK] AWS DynamoDB connection established
✅ Updated question ID 5 in DynamoDB
✅ Updated question ID 73 in DynamoDB
[CLOSED] AWS DynamoDB connection closed

✅ All questions updated in DynamoDB


## Step 6: Verify Changes

Review the updated questions to make sure they're correct.


In [10]:
# Show all YES/NO questions after inversion
yes_no_questions_updated = df[df["Scoring"] == "YES/NO"].copy()

print("="*80)
print("All YES/NO Questions (after inversion):")
print("="*80)

for idx, row in yes_no_questions_updated.iterrows():
    is_inverted = row["ID"] in QUESTIONS_TO_INVERT
    marker = "🔄 INVERTED" if is_inverted else "  "
    
    print(f"\n{marker} ID: {row['ID']} | RedFlag_ID: {row['RedFLag_ID']}")
    print(f"   Category: {row['Category_Name']}")
    print(f"   TR: {row['Question_TR']}")
    print(f"   EN: {row['Question_EN']}")
    print("-" * 80)

print("\n✅ Verification complete!")


All YES/NO Questions (after inversion):

   ID: 2 | RedFlag_ID: 2
   Category: Sosyal Kimlik
   TR: Prestijli ve köklü bir lise de mi okudu? Örneğin: Galatasaray, Kabataş, Üsküdar Amerikan, Kadıköy Anadolu, İstanbul Erkek, Robert..
   EN: Did he attend a prestigious and well-established high school? (e.g., Galatasaray, Kabataş, Üsküdar American, Kadıköy Anadolu, Istanbul Erkek, Robert, etc.)
--------------------------------------------------------------------------------

   ID: 4 | RedFlag_ID: 4
   Category: Sosyal Kimlik
   TR: Mesleği şunlardan birisi mi?: Doktor, Avukat, Dişhekimi, PT(personal trainer), Akademisyen, Sanatçı
   EN: Is his profession one of the following? Doctor, Lawyer, Dentist, Personal Trainer, Academic, Artist
--------------------------------------------------------------------------------

🔄 INVERTED ID: 5 | RedFlag_ID: 5
   Category: Sosyal Kimlik
   TR: De/da'ları ve gerektiğinde ki'yi ayrı yazamaz?
   EN: Does he have poor grammar?
---------------------------